# Healthy Streets of Los Angeles Mobility plan analysis

This project finds out how well LA Mobility Plan 2035 was implemented to this day (November 2023) for protected and unprotected bike lanes and protected bus lanes. Also it finds the same data for Neighborhood Enhanced Network bike lanes.

Sources:
* Current Bike Lanes: https://geohub.lacity.org/datasets/ladot::city-of-los-angeles-bikeways-1/explore
* Mobility Plan bike lanes: https://geohub.lacity.org/datasets/bicycle-enhanced-network/explore?location=34.020251%2C-118.411699%2C11.58 </br >
Mobility Plan bike lanes tiers are in "Bicycle_N" column
* Neighborhood Enhanced Network bike lanes https://geohub.lacity.org/datasets/lahub::neighborhood-enhanced-network/explore
* Current Bus Lanes: https://www.google.com/maps/d/u/1/edit?mid=10co-9X_jGrJtGBA9qVhwWOqUpQNng2dx&usp=sharing 
* Mobility plan Bus lanes: https://geohub.lacity.org/datasets/transit-enhanced-network/explore?location=34.018131%2C-118.376481%2C11.68

Compare: 
* Actual Bus Lanes = Mobility Plan tier 1
* Actual Bike Lanes Class 4 = Protected (Mobility Plan tier 1)
* Actual Bike Lanes Class 2 = Unprotected (Mobility Plan tier 2 and 3)
* Actual Bike Lanes Class 3 = Neighborhood Enhanced Network

Deliverable: 
* What % of the mobility plan has been implemented
* Geojson result
* year > 2015, because Mobility Plan was approved in late 2015

Any questions - elena.sunchugasheva@gmail.com

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import folium

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)

In [ ]:
def show_map(gdf1, name1, color1, gdf2, name2, color2):
    
    f = folium.Figure(width=1000, height=500)
    
    m = gdf1.explore(
        name = name1,
        color = color1
    ).add_to(f)

    map_2 = gdf2.explore(
        m=m,  # pass the map object
        name = name2,
        color = color2
    )

    folium.TileLayer(
        'CartoDB positron',
        show=False
    ).add_to(m) 

    folium.LayerControl().add_to(m)
    
    return m

## bike lanes

In [ ]:
bike_plan_file = open('Bicycle_Enhanced_Network.geojson')
bike_plan_geo_raw = gpd.read_file(bike_plan_file)
bike_actual_file = open('City_of_Los_Angeles_Bikeways.geojson')
bike_actual_geo_raw = gpd.read_file(bike_actual_file)

In [ ]:
bike_plan_geo = bike_plan_geo_raw.copy()
bike_actual_geo = bike_actual_geo_raw.copy()
print(
    'Mobility plan records:',
    bike_plan_geo.shape[0],
    '\nActual bike lanes records:',
    bike_actual_geo.shape[0],
    '\nActual lanes without construction year:',
    bike_actual_geo[bike_actual_geo.Year_.isnull()].shape[0]
)
display(bike_plan_geo.head(1))
display(bike_actual_geo.tail(1))

### Clean up data

 actual bike paths surprisingly have z coordinate, so following is the best way to get rid of it (all z==0) I came up with

In [ ]:
def get_xy(line):
    if line.has_z:
        xy_line = [xy[:2] for xy in list(line.coords)]
    return LineString(xy_line)

In [ ]:
bike_actual_geo = bike_actual_geo.explode(ignore_index=False, index_parts=False)
bike_actual_geo.geometry = bike_actual_geo.geometry.apply(lambda x: get_xy(x))
display(bike_actual_geo.head(1))

### Intersecting linestring data
As we see below, lines for planned and actual bike paths are not exactly the same (but actually close, see the map below).

In [ ]:
# show_map(
#     gdf1 = bike_actual_geo[['SECT_ID', 'geometry']],
#     name1 = 'actual', 
#     color1 = 'blue',
#     gdf2 = bike_plan_geo[['OBJECTID', 'geometry']],
#     name2 = 'plan', 
#     color2 = 'green',
# )

### Intersecting a linesting and polygon
Lets make a polygon with some width of actual data. We chose it over planned because:
1. we won't double count bike paths, planned data has one line where actual data sometines two (one bike lane one way and one lane the other way).
2. we're comparing result (implemented bike lanes) with planned data, so there we won't depend on possible irregularities of existing bike paths</br>

In [ ]:
def buffer(gdf, radius, proj='EPSG:4326', proj_calc='EPSG:3857'):
    '''
    convert a gdf of linestrings into a gdf of polygons with radius
    - gdf - GeoDataFrame, has column "geometry"
    - radius - radius of bufferm meters
    - proj - projection of the original dataset
    - proj_calc='EPSG:3857' by default
    '''
    gdf_calc = gdf.to_crs(proj_calc)
    #print('data proj:', proj, '\ncalculation proj: ', proj_calc)
    gdf['buffered'] = gdf_calc.buffer(radius, cap_style=2).to_crs(proj)
    gdf.set_geometry('buffered', inplace=True)
    
    # merge all intersecting buffered polygons
    gdf_return = gpd.GeoDataFrame(
        geometry=[gdf.unary_union]
    ).explode(
        index_parts=False
    ).reset_index(
        drop=True
    )
    gdf_return.geometry.crs = proj
    
    return gdf_return

In [ ]:
def get_length(gdf, column='geometry', proj='EPSG:4326', proj_calc='EPSG:3857'):
    '''
    convert a gdf of linestrings into a gdf of polygons with radius
    - gdf - GeoDataFrame
    - column - column to be used for length calculation if not "geometry"
    - proj - projection of the original dataset
    - proj_calc='EPSG:3857' by default
    '''
    if not column:
        gdf.set_geometry(column, inplace=True)
    # convert projection to proj_calc, if default - the units will be in meters
    gdf_m = gdf.to_crs(proj_calc)
    gdf['length_m'] = gdf_m.length
    length_m = gdf.length_m.sum()
    
    return length_m

In [ ]:
def compare_length(
        name,
        gdf_plan,
        gdf_actual,
        conditions,
        year = None,
        column_year = 'Year_',
        print_map = True,
        test_map = False
    ):
    '''
    - name 
    - gdf_plan - GeoDataFrame of Mobility plan
    - gdf_actual - GeoDataFrame of existing roads, has a column_year
    - conditions - filter specific types of bike/bus lanes
    - year - filter data after construction year
    - column_year - column of construction year
    - print_map - show final map
    - test_map - show interim map
    '''
    print(name)
    
    if year:
        gdf_actual = gdf_actual[gdf_actual[column_year]>year].copy()
    if conditions['plan']:
        if 'bike' in key:
            gdf_plan = gdf_plan[gdf_plan.BICYCLE_N.isin(
                conditions['plan']
            )].copy()
        if 'bus' in key:
            gdf_plan = gdf_plan[gdf_plan.TRANSIT_N.isin(
                conditions['plan']
            )].copy()
    if conditions['actual']:
        gdf_actual = gdf_actual[gdf_actual.Class.isin(
            conditions['actual']
        )].copy()
    
    # widen linestring to polygon
    gdf_actual_buffer = buffer(
        gdf_actual, 
        radius = 10,
        proj = gdf_actual.geometry.crs
    )
    if test_map:
        display(
            show_map(
                gdf2 = gdf_actual_buffer[['geometry']],
                name2 = 'implemented', 
                color2 = 'blue',
                gdf1 = gdf_plan[['OBJECTID', 'geometry']],
                name1 = 'plan', 
                color1 = 'green',
            )
        )
    
    # intersect polygons of actual and planned paths
    gdf_implemented = gpd.overlay(
        gdf_plan, 
        gdf_actual_buffer, 
        how='intersection',
        keep_geom_type=False
    )
    
    # map of buffered planned (green) and implemented (red) paths
    if print_map:
        display(
            show_map(
                gdf2 = gdf_implemented[['OBJECTID', 'geometry']],
                name2 = 'implemented', 
                color2 = 'red',
                gdf1 = gdf_plan[['OBJECTID', 'geometry']],
                name1 = 'plan', 
                color1 = 'green',
            )
        )
        
    print(
        'since', year, 'year:',
        '\nimplemented lanes records:', gdf_implemented.shape[0],
        '\nplanned lanes records:', gdf_plan.shape[0]        
    )
    
    percentage = round(
        get_length(gdf_implemented)/get_length(gdf_plan)*100, 
        2
    )

    return percentage, gdf_implemented

### calculate percentages

In [ ]:
bike_conditions = {
    'protected bike lane': {'plan': [1], 'actual': [4]},
    'unprotected bike lane': {'plan': [2, 3], 'actual': [2]},
}
results = []
result_gdfs = []

for key in bike_conditions.keys():
    for year in (None, 2015):
        percentage, result_gdf = compare_length(
            name = key,
            conditions = bike_conditions[key],
            gdf_actual = bike_actual_geo[[
                            'OBJECTID_12',
                            'SECT_ID',
                            'Class',
                            'Year_',
                            'geometry'
                        ]],
            gdf_plan = bike_plan_geo,
            print_map = True,
            test_map = False,
            year = year,
            column_year = 'Year_'
        )
        results.append([key, year, percentage])
        result_gdfs.append([key, year, result_gdf])
        
        print(
            percentage,
            f'% of {key} implemented after {year}',
            '\n'
        )
        
    print('\n----')    

### NEN vs class 3

In [ ]:
bike_nen_file = open('Neighborhood_Enhanced_Network.geojson')
bike_nen_geo_raw = gpd.read_file(bike_nen_file)

In [ ]:
bike_nen_geo = bike_nen_geo_raw.copy()
print(bike_nen_geo.shape[0])
display(bike_nen_geo.head())

In [ ]:
bike_nen_conditions = {
    'NEN bike lane': {'plan': None, 'actual': [3]},
}

for key in bike_nen_conditions.keys():
    for year in (None, 2015):
        percentage, result_gdf = compare_length(
            name = key,
            conditions = bike_nen_conditions[key],
            gdf_actual = bike_actual_geo[[
                            'OBJECTID_12',
                            'SECT_ID',
                            'Class',
                            'Year_',
                            'geometry'
                        ]],
            gdf_plan = bike_nen_geo,
            print_map = True,
            year = year,
            column_year = 'Year_',
            test_map = False
        )
        results.append([key, year, percentage])
        result_gdfs.append([key, year, result_gdf])
        
        print(
            percentage,
            f'% of {key} implemented after {year}',
            '\n'
        )
        
    print('\n----')    

## bus lanes

In [ ]:
bus_plan_file = open('Transit_Enhanced_Network.geojson')
bus_plan_geo_raw = gpd.read_file(bus_plan_file)
bus_actual_file = open('Bus_Only_Lanes_in_LA.geojson')
bus_actual_geo_raw = gpd.read_file(bus_actual_file)

In [ ]:
bus_plan_geo = bus_plan_geo_raw.copy()
bus_actual_geo = bus_actual_geo_raw.copy()
print(
    'Mobility plan records:',
    bus_plan_geo.shape[0],
    '\nActual bus lanes records:',
    bus_actual_geo.shape[0]
)
display(bus_plan_geo.head())
display(bus_actual_geo.head())

In [ ]:
# take a look at planned vs existing protected lanes
protected_bus_plan = bus_plan_geo[bus_plan_geo.TRANSIT_N==1].copy()
display(
            show_map(
                gdf2 = bus_actual_geo[['Name', 'geometry']],
                name2 = 'actual', 
                color2 = 'blue',
                gdf1 = protected_bus_plan[['OBJECTID', 'geometry']],
                name1 = 'plan', 
                color1 = 'green',
            )
        )

In [ ]:
bus_conditions = {
    'protected bus lane': {'plan': [1], 'actual': None}
}

for key in bus_conditions.keys():
    year = None
    percentage, result_gdf = compare_length(
        name = key,
        conditions = bus_conditions[key],
        gdf_actual = bus_actual_geo[[
                        'Name',
                        'geometry'
                    ]].copy(),
        gdf_plan = bus_plan_geo,
        print_map = True,
        test_map = True,
        year = year,
        column_year = None 
    )
    results.append([key, year, percentage])
    result_gdfs.append([key, year, result_gdf])
    
    print(
        percentage,
        f'% of {key} implemented after {year}',
        '\n'
    )
        
    print('\n----')    

In [ ]:
percentage_all = round(
        get_length(bus_actual_geo)/get_length(protected_bus_plan)*100, 
        2
    )

In [ ]:
results.append([
    'all bus lanes',
    None,
    percentage_all
])

## save results

In [ ]:
results_df = pd.DataFrame(
    columns=['transport type', 'year', 'implemented_perc'],
    data = results
)
display(results_df)

In [ ]:
results_df.to_csv('mobility_plan_percentage_implemented.csv', index=False)

In [ ]:
for df in result_gdfs:
    name = df[0].replace(' ', '_') + '_implemented'
    year = df[1]
    if year:
        name = name + f'_since_{year}'
    df[2].to_file(name + '.geojson', driver='GeoJSON')